In [63]:
import os
import openai
import matplotlib.pyplot as plt
import numpy as np
openai.api_key = "sk-LopWyMKqQ2aCySPMeFSZLyrAur0a8EACzgbHBXMK"
from sklearn.datasets import fetch_openml

from sklearn import tree


adult = fetch_openml(data_id=43436) ## adult

#adult = fetch_openml(data_id=196) ## mpg
#adult = fetch_openml(data_id=61) ## iris


In [64]:

adult['data'].columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [65]:
adult['data'].std(axis='index')

/tmp/ipykernel_14966/3250815263.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  adult['data'].std(axis='index')


age                   13.640433
fnlwgt            105549.977697
education.num          2.572720
capital.gain        7385.292085
capital.loss         402.960219
hours.per.week        12.347429
dtype: float64

In [62]:
mdatos=adult['data']
#remove missing
mdatos=mdatos.dropna()
miser=[float(x) for x in mdatos["model"]]
sum(miser)/len(miser)
import numpy as np

np.std(miser)

3.6790348996151727

In [19]:
#!/usr/bin/env python
# coding: utf-8

# In[107]:


import os
import openai
import matplotlib.pyplot as plt
import numpy as np
openai.api_key = "sk-LopWyMKqQ2aCySPMeFSZLyrAur0a8EACzgbHBXMK"
from sklearn.datasets import fetch_openml

from sklearn import tree


adult = fetch_openml(data_id=43436) ## adult

adult = fetch_openml(data_id=61) 

results=[]

# In[108]:


from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
mdatos=adult['data']
#remove missing
mdatos=mdatos.dropna()
#remove classes with few elements


#cl="race"
for cl in mdatos.columns:
  
  print("column:" +cl)
  
  classif=False
  if (mdatos[cl].dtype=="O"):
      classif=True
      datos=mdatos.groupby(cl).filter(lambda x: len(x) > 10)
  else:
      classif=False
      datos=mdatos

  datos = datos.sample(frac=1).reset_index(drop=True)
  X=datos.loc[:, datos.columns != cl] # Features
  y=datos[cl]
   
  
  
  if (classif): 
      X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
  else:
          X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
      
  
  
  # In[109]:
  
  
  from sklearn.pipeline import Pipeline
  from sklearn.preprocessing import OneHotEncoder
  from sklearn.impute import SimpleImputer
  from sklearn.compose import ColumnTransformer
  cols = datos.columns
  
  num_cols = X._get_numeric_data().columns
  categorical_columns  = [col for col in X.columns if X[col].dtype=="O"]
  
  
  numerical_columns = list(num_cols)
  
  #X = X[categorical_columns + numerical_columns]
  
  
  
  #categorical_columns = X._get_numeric_data().columns
  
  
  #X = X[categorical_columns + numerical_columns]
  
  #X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
  
  
  categorical_encoder = OneHotEncoder(handle_unknown="ignore")
  numerical_pipe = Pipeline([("imputer", SimpleImputer(strategy="mean"))])
  
  preprocessing = ColumnTransformer(
      [
          ("cat", categorical_encoder, categorical_columns),
          ("num", numerical_pipe, numerical_columns),
      ]
   )
  
  if (classif):
   rf = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeClassifier(random_state=42)),
      ]
   )
  else:
   rf = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeRegressor(random_state=42)),
      ]
   ) 
  
  
  rf.fit(X_train, y_train)
  
  
  # In[110]:
  
  
  
 # if (classif):
#    print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
#    print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))
#  else:
#    print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
#    print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))
  
  
  # In[111]:
  
  
  from sklearn.inspection import permutation_importance
  result = permutation_importance(
      rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
  )
  sorted_idx = result.importances_mean.argsort()
  
  #fig, ax = plt.subplots()
  #ax.boxplot(
  #    result.importances[sorted_idx].T, vert=False, labels=X_test.columns[sorted_idx]
  #)
  #ax.set_title("Permutation Importances (test set)")
  #fig.tight_layout()
  #plt.show()
  
  
  # In[112]:
  
  
  nsel=1
  selected=list(X_test.columns[sorted_idx][-nsel:])
  
  
  # In[113]:
  
  
  Xn=datos.loc[:, selected] # Features 
  
  
  # In[114]:
  
  
  
  num_cols = Xn._get_numeric_data().columns
  categorical_columns  = [col for col in Xn.columns if Xn[col].dtype=="O"]
  
  
  numerical_columns = list(num_cols)
  
  #X = X[categorical_columns + numerical_columns]
  
  
  
  categorical_encoder = OneHotEncoder(handle_unknown="ignore")
  numerical_pipe = Pipeline([("imputer", SimpleImputer(strategy="mean"))])
  
  preprocessing = ColumnTransformer(
      [
          ("cat", categorical_encoder, categorical_columns),
          ("num", numerical_pipe, numerical_columns),
      ]
   )
  
  if (classif):
   dt = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeClassifier(random_state=42)),
      ]
   )
  else:
   dt = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeRegressor(random_state=42)),
      ]
   ) 
  
  
  
  
  # In[115]:
  
  
  n=10 #repetitions
  tam=100 #prompt
  
  responses=[]
  correct=[]
  responsesg=[]
  
  for ij in range(n):
    #  print (ij)
     
      if (classif):
       X1_train, X1_test, y1_train, y1_test = train_test_split(Xn, y,train_size=tam, stratify=y, random_state=ij)
      else:
       X1_train, X1_test, y1_train, y1_test = train_test_split(Xn, y,train_size=tam, random_state=ij)
      ### DT
      dt.fit(X1_train, y1_train)
      predictions = dt.predict(X1_test)
      predictions[0]==y1_test.iloc[0]
      #print(predictions[0])
      #print(y1_test.iloc[0])
      correct.append(y1_test.iloc[0]) 
      responses.append(predictions[0])
      ### GPT3
      att=X1_train.columns[0]
      prompt=""
      #Build Prompt
      for i in range (0,(tam-1)):
              prompt=prompt+"input:"+str(X1_train[att].iloc[i])+', output:'+str(y1_train.iloc[i])+'\n'
      prompt1=prompt+" input:"+str(X1_test[att].iloc[0])+', output:'
      response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt1,
        temperature=0,
        max_tokens=64,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
          )
      #Response
     # if (response["choices"][0]["text"].split('\n')[0]==y1_test.iloc[0]):
     #     accg=accg+1
      responsesg.append(response["choices"][0]["text"].split('\n')[0])   
       
  
  
  # In[116]:
  
  
  acc=0
  accg=0
  if classif:
   for i in range(n):  
          if (responses[i]==correct[i]):
              acc=acc+1
          if (responsesg[i]==correct[i]):
              accg=accg+1
  
  if not(classif):
   for i in range(n):
          acc=acc+abs(responses[i]-float(correct[i]))
          accg=accg+abs(float(responsesg[i])-float(correct[i]))
               
  
  
  # In[117]:
  
  if classif:  
   print("accuracy gpt3")
   print(float(accg)/n)
   print("accuracy DT")
   print(float(acc)/n)
   results.append((cl,n,float(accg)/n,float(acc)/n))

  if not(classif):
    print("MAE gpt3")
    print(float(accg)/n)
    print("MAE DT")
    print(float(acc)/n)
    results.append((cl,n,float(accg)/n,float(acc)/n))

  
  
  
  


column:sepallength
MAE gpt3
0.26000000000000006
MAE DT
0.3041666666666667
column:sepalwidth
MAE gpt3
0.14000000000000004
MAE DT
0.12077489177489183
column:petallength
MAE gpt3
0.21000000000000005
MAE DT
0.22358730158730164
column:petalwidth
MAE gpt3
0.11000000000000001
MAE DT
0.14452380952380955


In [13]:
responses[i]-float(correct[i])

0.0

In [20]:
results


[('sepallength', 10, 0.26000000000000006, 0.3041666666666667),
 ('sepalwidth', 10, 0.14000000000000004, 0.12077489177489183),
 ('petallength', 10, 0.21000000000000005, 0.22358730158730164),
 ('petalwidth', 10, 0.11000000000000001, 0.14452380952380955)]

In [21]:
mdatos.columns

Index(['sepallength', 'sepalwidth', 'petallength', 'petalwidth'], dtype='object')

In [22]:
adult['data']

sepallength  sepalwidth  petallength  petalwidth
0            5.1         3.5          1.4         0.2
1            4.9         3.0          1.4         0.2
2            4.7         3.2          1.3         0.2
3            4.6         3.1          1.5         0.2
4            5.0         3.6          1.4         0.2
..           ...         ...          ...         ...
145          6.7         3.0          5.2         2.3
146          6.3         2.5          5.0         1.9
147          6.5         3.0          5.2         2.0
148          6.2         3.4          5.4         2.3
149          5.9         3.0          5.1         1.8

[150 rows x 4 columns]

AttributeError: iloc

In [49]:
import pandas as pd

df = pd.read_excel (r'UCS-Satellite-Database-1-1-2022.xls')
df = pd.read_csv('tennis.csv')
#df.mean(axis='index')
df.std(axis='index')


/tmp/ipykernel_14966/2963096531.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.std(axis='index')


birth_year       24.936523
birth_month       3.416293
birth_day         8.736898
turned_pro        9.833321
weight_lbs       30.090064
weight_kg        13.650145
height_inches     3.567250
height_cm         9.109566
dtype: float64